In [ ]:
!export CUDA_LAUNCH_BLOCKING=1

!pip install transformers
!pip install datasets fsspec gcsfs

INFO: pip is looking at multiple versions of gcsfs to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 11.8 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
  Attempting uninstall: gcsfs
    Found existing installation: gcsfs 2024.10.0
    Uninstalling gcsfs-2024.10.0:
      Successfully uninstalled gcsfs-2024.10.0


In [ ]:
from transformers import pipeline

from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from transformers import BlenderbotTokenizer, BlenderbotForConditionalGeneration
from transformers import DataCollatorForSeq2Seq

from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments

from transformers import EarlyStoppingCallback

import pandas as pd

from datasets import Dataset
import torch

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

data_dir = '/content/drive/My Drive/CMPE258DL/dataset/'
results_data_dir = '/content/drive/My Drive/CMPE258DL/'

csv_path = data_dir + "balanced_training_dataset.csv"
training_data = pd.read_csv(csv_path)

csv_path = data_dir + "validation_dataset.csv"
validation_data = pd.read_csv(csv_path)

csv_path = data_dir + "test_dataset.csv"
test_data = pd.read_csv(csv_path)

model_save_path = "/content/drive/My Drive/CMPE258DL/fine_tuned_blenderbot_5_epochs_final"

print(test_data['instruction'].isna().sum())  # Count NaN values
print((test_data['instruction'] == "").sum())  # Count empty strings

print(training_data.dtypes)
print(training_data[['instruction', 'output']].head())

Mounted at /content/drive
0
0
instruction    object
output         object
topics         object
dtype: object
                                         instruction  \
0  I'm having trouble managing my anger. What can...   
1  I'm having trouble communicating with my partn...   
2   I'm having trouble with my anger. What can I do?   
3  I'm having trouble with my own behavior and fe...   
4  I'm having trouble forgiving someone who hurt ...   

                                              output  
0  It's important to identify triggers that cause...  
1  Communication is key in any relationship. Let'...  
2  Anger can be a challenging emotion to manage, ...  
3  Anger management is a common concern for many ...  
4  Consider seeking therapy to work through any u...  


In [ ]:
# Check for GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cpu


In [ ]:
# Load the model and tokenizer
# fine_tuned_model = BlenderbotForConditionalGeneration.from_pretrained(model_save_path)
# fine_tuned_tokenizer = BlenderbotTokenizer.from_pretrained(model_save_path)

# print("Fine-tuned model and tokenizer loaded successfully!")

In [ ]:
mname = "facebook/blenderbot-400M-distill"
tokenizer = BlenderbotTokenizer.from_pretrained(mname)
model = BlenderbotForConditionalGeneration.from_pretrained(mname)

# Ensure consistent max_length
tokenizer.model_max_length = 512
model.config.max_length = 256  # Maximum length for generation

# Check if the tokenizer already has a pad token
if tokenizer.pad_token is None:
   print("Adding padding token '<pad>' to the tokenizer.")
   tokenizer.add_special_tokens({'pad_token': '<pad>'})  # Add <pad> token if missing

# Resize model embeddings to match the tokenizer's vocabulary size
model.resize_token_embeddings(len(tokenizer))
print(f"Resized model embeddings to match tokenizer vocab size: {len(tokenizer)}")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.15k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/127k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/62.9k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/16.0 [00:00<?, ?B/s]

...

config.json:   0%|          | 0.00/1.57k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/730M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/347 [00:00<?, ?B/s]

The new embeddings will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`


Resized model embeddings to match tokenizer vocab size: 8009


In [ ]:
def tokenize(row):
    # Convert instruction and output to strings
    instruction = str(row['instruction']) if pd.notna(row['instruction']) else ""
    output = str(row['output']) if pd.notna(row['output']) else ""

    # Tokenize the instruction and output
    inputs = tokenizer(instruction, truncation=True, padding="max_length", max_length=128, return_tensors="pt")
    targets = tokenizer(output, truncation=True, padding="max_length", max_length=128, return_tensors="pt")

    labels = targets["input_ids"].squeeze(0)
    labels[labels == tokenizer.pad_token_id] = -100  # Ignore padding in loss calculation

    return {"input_ids": inputs["input_ids"].squeeze(0).tolist(), "attention_mask": \
            inputs["attention_mask"].squeeze(0).tolist(), "labels": labels.tolist()}

# Tokenizing the datasets
tokenized_training_data = [tokenize(row) for row in training_data.to_dict("records")]
tokenized_validation_data = [tokenize(row) for row in validation_data.to_dict("records")]
tokenized_test_data = [tokenize(row) for row in test_data.to_dict("records")]

# Convert tokenized data to Dataset objects
train_dataset = Dataset.from_pandas(pd.DataFrame(tokenized_training_data))
eval_dataset = Dataset.from_pandas(pd.DataFrame(tokenized_validation_data))
test_dataset = Dataset.from_pandas(pd.DataFrame(tokenized_test_data))

print(train_dataset)
print(eval_dataset)
print(test_dataset)

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 3000
})
Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 1000
})
Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 1000
})


In [ ]:
#Trainer arguments
training_args = Seq2SeqTrainingArguments(
    output_dir="./results",          # Output directory for model checkpoints
    eval_strategy="epoch",           # Evaluate after each epoch
    save_strategy="epoch",           # Save model at the end of each epoch
    learning_rate=5e-5,              # Learning rate
    per_device_train_batch_size=1,   # Batch size for training
    per_device_eval_batch_size=1,    # Batch size for evaluation
    weight_decay=0.01,               # Weight decay
    save_total_limit=3,              # Total number of checkpoints saved
    num_train_epochs=1,              # Number of training epochs
    predict_with_generate=True,      # Use generate during evaluation
    fp16=True,                       # Use mixed precision
    logging_dir='./logs',            # Log
    load_best_model_at_end=True      # Load the best model at the end of training
)

# Trainer
trainer = Seq2SeqTrainer(model=model, args=training_args, train_dataset=train_dataset, \
                         eval_dataset=test_dataset, tokenizer=tokenizer, \
                         #callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]  # Early stopping with patience=3
)

# Train the model
trainer.train()
print("After training")
#!nvidia-smi

# Save fine-tuned model and tokenizer
model.save_pretrained(model_save_path)
tokenizer.save_pretrained(model_save_path)

print(f"Model and tokenizer saved to {model_save_path}")

<ipython-input-9-182e4fd78144>:19: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


Epoch,Training Loss,Validation Loss


In [ ]:
# Get log history
log_history = trainer.state.log_history

# Separate training and validation loss
training_loss = [entry["loss"] for entry in log_history if "loss" in entry]
validation_loss = [entry["eval_loss"] for entry in log_history if "eval_loss" in entry]
steps = [entry["step"] for entry in log_history if "loss" in entry]

# Plot the losses
plt.figure(figsize=(10, 6))
plt.plot(range(len(training_loss)), training_loss, label="Training Loss")
plt.plot(range(len(validation_loss)), validation_loss, label="Validation Loss", linestyle="--")
plt.xlabel("Steps (or Epochs for Validation)")
plt.ylabel("Loss")
plt.title("Training and Validation Loss Curves (Blenderbot 5 epochs)")
plt.legend()
plt.grid()
plt.savefig('Train_val_loss_curve.png')
plt.show()

In [ ]:
# Create a copy of the test data
test_data_with_responses = test_data.copy()

def generate_response(instruction):
    # Truncate input if it exceeds the model's maximum length
    if len(instruction) >= tokenizer.model_max_length:
        instruction = instruction[:tokenizer.model_max_length - 2]

    inputs = tokenizer(
        instruction,
        truncation=True,
        padding="max_length",
        max_length=128,
        return_tensors="pt"
    )
    # Move input tensors to the same device as the model
    inputs = {key: value.to(device) for key, value in inputs.items()}
    # Generate the response with adjusted parameters
    reply_ids = model.generate(
        **inputs,
        temperature=0.4,
        max_length=256,
        num_beams=5,
        early_stopping=True,
        eos_token_id=tokenizer.eos_token_id,
    )
    response = tokenizer.decode(reply_ids[0], skip_special_tokens=True)
    return response

# Apply response generation to test data
def generate_response_with_logging(instruction):
    try:
        if not isinstance(instruction, str) or len(instruction.strip()) == 0:
            raise ValueError("Invalid input: Input must be a non-empty string.")
        # Check input length before processing
        if len(instruction) > tokenizer.model_max_length - 2:
            print(f"Input too long, truncating: {instruction[:50]}...")
            instruction = instruction[:tokenizer.model_max_length - 2]
        print(f"Processing input of length {len(instruction)}")
        inputs = tokenizer(instruction, return_tensors="pt", padding="max_length", max_length=128, truncation=True)
        print(f"Tokenized inputs: {inputs}")
        return generate_response(instruction)
    except Exception as e:
        print(f"Error with input: {instruction[:50]}... -> {e}")
        return "Error"

test_data_with_responses['model_response'] = test_data_with_responses['instruction'].apply(generate_response_with_logging)

In [ ]:
# Save the updated test dataset
output_path = results_data_dir + "test_data_with_responses.csv"
test_data_with_responses.to_csv(output_path, index=False)
print("Responses saved to 'test_data_with_responses.csv'")